In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
agg3 =  pd.read_csv('./mid/agg3.txt', sep='\t')
merged_references_990 = pd.read_csv('./reference/merged_references_0_to_990.txt',  sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 15:21:25.552103
종료: 2024-02-20 15:32:05.699373


In [4]:
agg3

,uid,cluster_id,pubyear2
0,WOS:000239031100003,1333,2006
1,WOS:000234737300014,846,2006
2,WOS:000241844400003,997,2006
3,WOS:000417059300001,151,2006
4,WOS:000235792300010,210,2006
...,...,...,...
24339950,WOS:000750113400001,2719,2021
24339951,WOS:000839344100002,1636,2022
24339952,WOS:000798801800001,3584,2021
24339953,WOS:000763296900001,2034,2021


In [5]:
merged_references_990

,uid,ref_uid,ref_count
0,WOS:000201977400052,WOS:000201977400052.1,1
1,WOS:000200451400481,WOS:000200451400481.4,5
2,WOS:000200451400481,WOS:000200451400481.2,5
3,WOS:000200451400481,WOS:000200451400481.1,5
4,WOS:000200451400481,WOS:000200451400481.5,5
...,...,...,...
1001999995,WOS:000331178400067,WOS:000184382700010,15
1001999996,WOS:000331178400067,WOS:000274675000001,15
1001999997,WOS:000325157800001,WOS:A1991GL69600062,59
1001999998,WOS:000325157800001,WOS:A1977DZ86400001,59


In [5]:
# 각 컬럼별 null 값의 개수 계산
null_counts = agg3.isnull().sum()

# 결과 출력
print(null_counts)

uid           0
cluster_id    0
pubyear2      0
dtype: int64


조인하기

In [6]:
now = datetime.datetime.now() 
print('시작:', now)

third_result1 = pd.merge(merged_references_990,agg3, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 15:32:05.725562
종료: 2024-02-20 15:35:27.777133


In [7]:
# 각 컬럼별 null 값의 개수 계산
null_counts2 = third_result1.isnull().sum()

# 결과 출력
print(null_counts2)

uid                   0
ref_uid           83255
ref_count             0
cluster_id    690648997
pubyear2      690648997
dtype: int64


In [12]:
# 각 컬럼별 null 값의 개수 계산
null_counts00 = merged_references_990.isnull().sum()

# 결과 출력
print(null_counts00)

uid              0
ref_uid      83255
ref_count        0
dtype: int64


In [8]:
# 컬럼명 변경
third_result1 = third_result1.rename(columns={'ref_uid': 'refs_uid', 'ref_count': 'refs_n'})


# 컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id','pubyear2', 'refs_uid', 'refs_n']

# 새 순서로 DataFrame을 재구성
third_result2 = third_result1[new_order]



In [9]:
# 정수화
third_result2['cluster_id'] = third_result2['cluster_id'].astype('Int64')
third_result2['pubyear2'] = third_result2['pubyear2'].astype('Int64')

In [10]:
third_result2

,uid,cluster_id,pubyear2,refs_uid,refs_n
0,WOS:000201977400052,<NA>,<NA>,WOS:000201977400052.1,1
1,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.4,5
2,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.2,5
3,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.1,5
4,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.5,5
...,...,...,...,...,...
1001999995,WOS:000331178400067,<NA>,<NA>,WOS:000184382700010,15
1001999996,WOS:000331178400067,<NA>,<NA>,WOS:000274675000001,15
1001999997,WOS:000325157800001,25,2013,WOS:A1991GL69600062,59
1001999998,WOS:000325157800001,25,2013,WOS:A1977DZ86400001,59


refs_uid가 null인 값만 추출해서 엑셀로 저장해서 까보기

In [14]:
now = datetime.datetime.now() 
print('시작:', now)

# 'refs_uid' 컬럼에서 결측값(null)을 포함하는 행만 필터링
null_refs_uid_rows = third_result2[third_result2['refs_uid'].isnull()]

# 결측값을 포함하는 행 출력
#print(null_refs_uid_rows)

# 결과를 엑셀 파일로 저장
null_refs_uid_rows.to_excel('./check/null_refs_uid_rows_third_result2.xlsx', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 15:49:25.318052
종료: 2024-02-20 15:50:13.192538


In [15]:
# 결측값을 포함하는 행 출력
print(null_refs_uid_rows)

                            uid  cluster_id  pubyear2 refs_uid  refs_n
31137       WOS:000201402400076        <NA>      <NA>      NaN      17
64549       WOS:000200938100005        <NA>      <NA>      NaN      31
88398       WOS:000201276100003        <NA>      <NA>      NaN      12
145171      WOS:000201817500008        <NA>      <NA>      NaN      20
166852      WOS:000200172000004        <NA>      <NA>      NaN       5
...                         ...         ...       ...      ...     ...
1001973588  WOS:000327446500028           4      2013      NaN      66
1001978739  WOS:000324536800004         934      2013      NaN      57
1001979425  WOS:000323412600015          57      2013      NaN      42
1001985683  WOS:000322694400007        1604      2013      NaN      32
1001986358  WOS:000315573100008        1149      2013      NaN      52

[83255 rows x 5 columns]


ut_to_uid_is_core_set.pkl => P에서 ut를 통해 ut_to_uid(=uid)를 만듦 -> core가 1인 ut_to_uid(uid)만 추출한 set임.
merged_references의 uid가 이 pkl에 있다면(P에 존재하고 CORE가 1임) within_core 가 1, 아님 0으로 값 조정.

In [13]:
import pickle

# pickle 파일로부터 set 데이터 불러오기
with open('./mid/ut_to_uid_is_core_set.pkl', 'rb') as file:
    loaded_is_core_set = pickle.load(file)

#print(loaded_ut_ut_to_uid_is_core_set)
print(f"ut_to_uid 컬럼 데이터의 set 크기: {len(loaded_is_core_set)}")

ut_to_uid 컬럼 데이터의 set 크기: 22379399


refs_uid에 결측값이 존재함. 0과 1을 정수형으로 저장할 수 없음. 따라서 within_core의 데이터타입을 문자열로 지정함. 

In [17]:
import numpy as np


now = datetime.datetime.now() 
print('시작:', now)

# third_result2 DataFrame에 within_core 컬럼 추가 및 값 할당, 문자열로 처리
third_result2['within_core'] = third_result2['refs_uid'].apply(lambda x: 'NaN' if pd.isnull(x) else ('1' if x in loaded_is_core_set else '0'))

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 16:20:10.356284
종료: 2024-02-20 16:31:16.006772


within_core의 데이터 값 조사, 카운트 

In [18]:
now = datetime.datetime.now() 
print('시작:', now)


# within_core 컬럼의 데이터 값 조사 및 카운트
within_core_counts = third_result2['within_core'].value_counts()

# 결과 출력
print(within_core_counts)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 16:31:16.011596
within_core
0      927165537
1       74751208
NaN        83255
Name: count, dtype: int64
종료: 2024-02-20 16:31:44.506611


괜찮으면 : within_core 는 0과 1로만 이루어진 수.
저장하기 -> final폴더에 .

In [19]:

now = datetime.datetime.now() 
print('시작:', now)

third_result2.to_csv('./mid/third/third_result2.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-20 16:31:44.515588
종료: 2024-02-20 17:05:15.722909


In [20]:
third_result2

,uid,cluster_id,pubyear2,refs_uid,refs_n,within_core
0,WOS:000201977400052,<NA>,<NA>,WOS:000201977400052.1,1,0
1,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.4,5,0
2,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.2,5,0
3,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.1,5,0
4,WOS:000200451400481,<NA>,<NA>,WOS:000200451400481.5,5,0
...,...,...,...,...,...,...
1001999995,WOS:000331178400067,<NA>,<NA>,WOS:000184382700010,15,0
1001999996,WOS:000331178400067,<NA>,<NA>,WOS:000274675000001,15,1
1001999997,WOS:000325157800001,25,2013,WOS:A1991GL69600062,59,0
1001999998,WOS:000325157800001,25,2013,WOS:A1977DZ86400001,59,0
